In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This code is heavily borrowed from https://www.kaggle.com/code/yasufuminakama/nbme-deberta-base-baseline-train/notebook. Please upvote his workbook if you find this useful

I have given a very beginner level explanation of how the code works to find incorrect annotations, which is an important part of cleaning the data. 

**Basic Idea**

We loop through the offset values to find whether the annotation location is matching with any one of the offset values in the offset value list, if not it means there is some sort of error in the annotation

**Offset value** : Starting and ending value of each token

In [ ]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
# import torch.nn as nn
# from torch.nn import Parameter
# import torch.nn.functional as F
# from torch.optim import Adam, SGD, AdamW
# from torch.utils.data import DataLoader, Dataset

os.system('pip install transformers')
#os.system('python -m pip install --no-index --find-links=../input/nbme-pip-wheels transformers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Loading

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/nbme-score-clinical-patient-notes/train.csv')
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
features = pd.read_csv('../input/nbme-score-clinical-patient-notes/features.csv')
patient_notes = pd.read_csv('../input/nbme-score-clinical-patient-notes/patient_notes.csv')



In [ ]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

train['annotation_length'] = train['annotation'].apply(len)

## Undestanding Offset mapping

In [ ]:
model = "microsoft/deberta-base" 
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
tokenizer("Hello how are you", return_offsets_mapping=True)['offset_mapping']

As we can see offset mapping gives us indices of each token

(0,5) - Hello

(5,9) - how

(9,13) - are

(13,17) - you


If the annotation is a single word, it should exactly match one of the tuples in the offset mapping list

On the other hand, if annotation is a group of words, the starting index should match first element of one of the tuples and ending index should match second element of another tuple in the offset mapping list. This logic is implemented in the code below

## Finding Incorrect Annotations

In [ ]:
#https://huggingface.co/docs/transformers/preprocessing
#https://huggingface.co/course/chapter2/6?fw=pt
incorrect_annotation_ids = []
tk0 = tqdm(range(len(train)), total=len(train))
for i in tk0:
    
    text = train.loc[i, 'pn_history']
    feature_text = train.loc[i, 'feature_text']
    annotation_length = train.loc[i, 'annotation_length']
    annotation_list = train.loc[i, 'annotation']
    location_list = train.loc[i, 'location']

    encoded_text = tokenizer(text,
                             add_special_tokens=True,
                             max_length=512,
                             padding="max_length",
                             return_offsets_mapping=True)
    
   
    if annotation_length != 0:
       
        for location, annotation in zip(location_list, annotation_list):
            
            results = []
            #print(annotation)
            for loc in [s.split() for s in location.split(';')]:
                
                start_idx = -1
                end_idx = -1
                start, end = int(loc[0]), int(loc[1])
                #print(start, end)
                for idx in range(len(encoded_text['offset_mapping'])):
                    #if start value of annotation 
                    if (start_idx == -1) & (start < encoded_text['offset_mapping'][idx][0]):
                        start_idx = idx - 1
                    if (end_idx == -1) & (end <= encoded_text['offset_mapping'][idx][1]):
                        end_idx = idx
                    #print(start_idx," ", end_idx)
                if start_idx == -1:
                    start_idx = end_idx
                #print(start_idx, "--", end_idx)
                res = f"{encoded_text['offset_mapping'][start_idx][0] + 1} {encoded_text['offset_mapping'][end_idx][1]}"
                results.append(res)
            result = ";".join(results)
#             if (int(location.split()[0]) >= int(result.split()[0]) + 2 ) or (location == result and location[0] != '0'):
#                 print(i)
#                 print(location, result)
            if location != result:
                incorrect_annotation_ids.append(i)
               
    
            
            
            


In [ ]:
len(incorrect_annotation_ids)

In [ ]:
#incorrect_annotation_ids # Scope for improvement by removing punctuations

In [ ]:
num = 338

train.loc[num,]['pn_history']

In [ ]:
train.loc[num,]